# Check contents of the MS data in casatestdata

When writing new unittests, it is often a bit combersome to figure out if there is suitable MS data already
exists in casatestdata. 
This notebook scan subset of the directory contains MS data in casatestdata (either /home/casa or local copy)
to list some basic parameters (nchan, nspw, nrows, frequencies, etc)
***

# Install and Import

use local modular casa installion along with jupyter notebook and pandas

    export PPY=`which python3`
    virtualenv -p $PPY --setuptools ./local_python3
    ./local_python3/bin/pip install --upgrade pip
    ./local_python3/bin/pip install --upgrade numpy matplotlib ipython astropy
    ./local_python3/bin/pip install --extra-index-url https://casa-pip.nrao.edu/repository/pypi-group/simple casatools
    ./local_python3/bin/pip install --extra-index-url https://casa-pip.nrao.edu/repository/pypi-group/simple casatasks
    ./local_python3/bin/pip3 install jupyter
    ./local_python3/bin/pip3 install pandas


***

In [1]:
pwd

'/export/home/murasame/casawhl/casanb'

__Import Libraries__

In [3]:
# Import required tools/tasks
from casatools import simulator, image, table, coordsys, measures, componentlist, quanta, ctsys, ms, msmetadata
from casatasks import tclean, ft, imhead, listobs, exportfits, flagdata, bandpass, applycal
from casatasks.private import simutil
from casatools import msmetadata as msmd

import os
import pylab as pl
import numpy as np
#from astropy.io import fits
#from astropy.wcs import WCS
from ipywidgets import interact

# Instantiate all the required tools
#sm = simulator()
ia = image()
tb = table()
cs = coordsys()
me = measures()
qa = quanta()
ms = ms()
msmd = msmetadata()

#cl = componentlist()
#$mysu = simutil.simutil()

In [4]:
datadir='/export/home/murasame/casatestdata/measurementset/'

In [5]:
import glob
diroptionfull=glob.glob(datadir+'*')
diroption=[os.path.basename(x) for x in diroptionfull]
print(diroption)

['evla', 'almasd', 'other', 'vla', 'alma']


In [7]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
# defining some widgets
text = widgets.Text(
       value='My Text',
       description='Title', )
calendar = widgets.DatePicker(
           description='Select Date')
slider = widgets.FloatSlider(
         value=1,
         min=0,
         max=10.0,
         step=0.1,)
menu = widgets.Dropdown(
       options=diroption,
       value=diroption[0],
       description=datadir)
checkbox = widgets.Checkbox(
           description='Check to invert',)


In [8]:
menu

Dropdown(description='/export/home/murasame/casatestdata/measurementset/', options=('evla', 'almasd', 'other',…

In [18]:
if os.path.exists('runs'):
    os.system('rm -rf runs')
os.system('mkdir runs')

0

In [19]:
datadir

'/export/home/murasame/casatestdata/measurementset/'

In [20]:
datalist=glob.glob(datadir+menu.value+'/*')

In [21]:
for vis in datalist:
    visbase = os.path.basename(vis)
    try:
        listobs(vis,listfile='runs/'+visbase+'.listobs.txt')
    except:
        print('{} is not valid MS'.format(vis))

/export/home/murasame/casatestdata/measurementset/alma/venus_sf_ephem_test.residual is not valid MS
/export/home/murasame/casatestdata/measurementset/alma/venus_mos_ephem_test.residual is not valid MS


In [22]:
def getmssummary(vis):
    # nspw, nchans, fields, nrows, ncorr
    nchans=[]
    chan0=[]
    nrow=0
    nspw=0
    fields=[]
    nant=0
    msname=os.path.basename(vis)
    try:
        msmd.open(vis)
        mssummary = msmd.summary()
        nspw = msmd.nspw()
        for ispw in range(nspw):
            nchans.append(msmd.nchan(ispw))
            chan0.append(list(msmd.chanfreqs(ispw,unit='GHz'))[0])
        obs = msmd.observatorynames()[0]
        fields = list(mssummary['fields'])
        nrows = mssummary['nrows']
        nant = msmd.nantennas()
        msmd.close()
        return (obs, nrows, nspw,nchans, chan0, fields, nant)
    except:
        print("%s is not MS. Skip this file" % vis)
        return (0,0,0,0,0,0,0)

In [23]:
# create lists 
obsnamelist=[]
nrowslist=[]
nspwlist=[]
nchanlist=[]
chan0list=[]
fieldslist=[]
msnamelist=[]
nantlist=[]

for vis in datalist:
    (obs, nrows, nspw, nchans, chan0, fields, nant) = getmssummary(vis)
    if obs:
        msname = os.path.basename(vis)
        # creat lists
        msnamelist.append(msname)
        nrowslist.append(nrows)
        nspwlist.append(nspw)
        nchanlist.append(nchans)
        chan0list.append(chan0)
        fieldslist.append(fields)
        nantlist.append(nant)
        #print(msname, nspw, nchans, chan0, fields)

/export/home/murasame/casatestdata/measurementset/alma/venus_sf_ephem_test.residual is not MS. Skip this file
/export/home/murasame/casatestdata/measurementset/alma/venus_mos_ephem_test.residual is not MS. Skip this file


In [24]:
ls

casa-20211103-183124.log  casa-20211122-174129.log  list_testdata.ipynb
casa-20211103-221643.log  casa-20211122-174353.log  runs/
casa-20211103-224733.log  casa-20220119-203653.log  untitled
casa-20211105-221459.log  casa-20220119-224713.log


In [99]:
def make_clickable(fname):
    return f'<a href="{fname}">"{fname}</a>'

In [25]:
import pandas as pd
from IPython.display import HTML 
df = pd.DataFrame({'msname':msnamelist,'nrows':nrowslist, 'nspw':nspwlist,'nchan':nchanlist,'chan0 (GHz)':chan0list,'fields':fieldslist,'nant':nantlist} )
pd.set_option('display.max_rows', df.shape[0]+1)
pd.set_option('display.max_colwidth',None)
#df.style.format({"msname":make_clickable})
df['msname'] = df['msname'].apply(lambda x: f'<a target="_blank", href="runs/{x}.listobs.txt">{x}</a>')
#display(df)
HTML(df.to_html(escape=False))
#df['msname']

,msname,nrows,nspw,nchan,chan0 (GHz),fields,nant
0,uid___A002_Xbf792a_X26ec.ms,2,66,"[1, 1, 1, 1, 4, 128, 1, 128, 1, 128, 1, 128, 1, 1, 1, 1, 1, 128, 1, 128, 1, 128, 1, 128, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[282.994575, 284.932075, 294.994575, 296.994575, 184.55, 283.9867625, 282.9711375, 285.9242625, 284.9086375, 294.0023875, 294.9711375, 296.0023875, 296.9711375, 675.994575, 677.994575, 679.994575, 681.932075, 675.0023875, 675.97895, 677.0023875, 677.97895, 679.0023875, 679.97895, 680.9398875, 681.91645, 184.603, 184.532, 184.55, 184.53300000000004, 184.621, 184.576, 184.55200000000002, 184.6, 184.55, 184.521, 184.60500000000002, 184.518, 184.55, 184.588, 184.56300000000002, 184.535, 184.601, 184.586, 184.573, 184.521, 184.55, 184.529, 184.536, 184.55, 184.579, 184.585, 184.58700000000002, 184.566, 184.55, 184.55, 184.51494213867187, 184.53277783203126, 184.565, 184.578, 184.578, 184.559, 184.55, 184.59300000000002, 184.55, 184.55700000000002, 184.591]","[J1924-2914, J1957-3845, J2056-3208, AU_Mic, AU_Mic, AU_Mic]",41
1,wvrgcal4quasar_10s.ms,612,27,"[4, 128, 1, 128, 1, 128, 1, 128, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[184.55, 86.29811749989999, 85.28249249989999, 88.17311749989999, 87.15749249989999, 96.3137425, 97.2824925, 98.1887425, 99.1574925, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55, 184.55]","[0132-169, 0455-462, 0530+135, 0334-401, 0538-440, 0208-512, 0506-610, 0522-364, 0423-013, 0607-085, 0510+180, 0750+125, 0336+323, 0428-379, 3c120, 0730-116]",18
2,twhya.short.ms,5940,4,"[3840, 3840, 3840, 3840]","[356.4979360351563, 357.73431396494374, 346.0343139647438, 343.95593603515624]","[3c279, Titan, TW Hya, J1147-382=QSO, J1037-295=QSO]",9
3,X425.pm04.scan4.ms,60,1,[4080],[101.94584960927499],[M100],1
4,uid___A002_X72c4aa_X8f5_scan21_spw18_field2_corrXX.ms,39150,1,[1],[289.2006288909753],[J1058+0133],29
5,CAS-4850-30s-limit-ALMA.ms,36000,1,[100],[321.1782494501964],[Vy_CMa],16
6,venus_ephem_test.ms,157920,1,[1],[245.24981349430084],"[Venus, Venus, Venus, Venus, Venus, Venus, Venus]",47
7,nep3-shrunk.ms,5940,1,[9],[340.76240625],"[J0006-0623, J2148+0657, Pallas, J2246-1206, Neptune]",39
8,uid___A002_X8ca70c_X5_shortened.ms,250560,9,"[4, 64, 1, 64, 1, 64, 1, 64, 1]","[184.55, 86.29030499989999, 85.29030499989999, 88.16530499989999, 87.16530499989999, 96.321555, 97.290305, 98.196555, 99.165305]","[J1617-7717, J1423-7829]",29
9,M51.ms,4320,1,[1],[668.0],[M51_0],16
